# Generate the evaluation table with financial metrics

In [0]:
%run "../utils/mount_configuration"

In [0]:
# Load evaluation date from 'tickers_quality_evaluation' task
evaluation_date = dbutils.jobs.taskValues.get(taskKey="tickers_quality_evaluation", key="evaluation_date", debugValue="2025-03-17")

In [0]:
from pyspark.sql.functions import col

warren_buffet_stocks_selection_df = spark.read \
    .format('delta') \
    .load(f'{presentation_folder_path}/warren_buffet_stocks_selection') \
    .filter(col('date') == evaluation_date)

# display(warren_buffet_stocks_selection_df)

In [0]:
benjamin_graham_stocks_selection_df = spark.read \
    .format('delta') \
    .load(f'{presentation_folder_path}/benjamin_graham_stocks_selection') \
    .filter(col('date') == evaluation_date)

# display(benjamin_graham_stocks_selection_df)

In [0]:
# Creation directory in which will be stored all files for each evalution date 
directory = f'{presentation_folder_path}/investment_recommendations/{evaluation_date}'
dbutils.fs.mkdirs(directory)

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


# Save the selected stocks table as an image with customized styling and formatting.
def save_table_as_image(stocks_selection, investment_philosophy):

    #Basic configs
    sns.set(style="whitegrid")
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.axis('off')  

    #Table configs
    table = ax.table(cellText=stocks_selection.values, colLabels=stocks_selection.columns, cellLoc='center', loc='center')
    table.auto_set_font_size(False)
    table.set_fontsize(12)
    table.scale(1.2, 1.2)
    table.auto_set_font_size(True)

    # Adjusting column widths (based on text length in headers)
    for i, col in enumerate(stocks_selection.columns):
        max_len = max(stocks_selection[col].astype(str).apply(len).max(), len(col))  
        table.auto_set_column_width([i]) 
        table[0, i].set_fontsize(14) 

    # Coloring the background for headers and rows
    for (i, j), cell in table.get_celld().items():
        if i == 0:  
            cell.set_fontsize(16)
            cell.set_text_props(weight='bold')  
            cell.set_facecolor('#FFEB99')  
        elif i % 2 == 0:  
            cell.set_facecolor('#FFF4E6')  
        else:  
            cell.set_facecolor('#FFFAF1')

    #Define filename and whole filepath for the chart
    plot_filename = f'{investment_philosophy}_stocks_selection.png'
    plot_filepath = f'/dbfs/{directory}/{plot_filename}'
    plt.savefig(plot_filepath, bbox_inches='tight', dpi=300)
    plt.show()
    plt.close()

In [0]:

save_table_as_image(warren_buffet_stocks_selection_df.toPandas(), 'warren_buffet')
save_table_as_image(benjamin_graham_stocks_selection_df.toPandas(), 'benjamin_graham')